In [1]:
import os

In [2]:
%pwd

'D:\\Machine_Learning\\Portfolio_Project_one\\Experimentation'

In [3]:
os.chdir('../')

In [4]:
%pwd

'D:\\Machine_Learning\\Portfolio_Project_one'

In [6]:
#  Entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [8]:
#  Cpnfiguration Manager
from portfolio_project_one.constants import *
from portfolio_project_one.utils.common import read_yaml, create_directories

class ConfigurationManager:                  
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,                     # These were all defined in constants
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifact_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            alpha= params.alpha,
            l1_ratio= params.l1_ratio,
            target_column= schema.name
        )

        return model_trainer_config

In [11]:
#  Components

import pandas as pd
import os
from portfolio_project_one import logger
from sklearn.linear_model import ElasticNet
import joblib

class ModelTrainer:

    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop(columns=self.config.target_column)
        X_test = test_data.drop(columns=self.config.target_column)
        y_train = train_data[self.config.target_column]
        y_test = test_data[self.config.target_column]

        model = ElasticNet(alpha= self.config.alpha, l1_ratio= self.config.l1_ratio)
        model.fit(X_train, y_train)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        

In [12]:
#  pipeline

try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config= model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise e
    

[2023-10-08 16:19:47,262: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2023-10-08 16:19:47,282: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-10-08 16:19:47,283: INFO: common: yaml file: schema.yaml loaded succesfully]
[2023-10-08 16:19:47,294: INFO: common: Created directory at : artifacts]
[2023-10-08 16:19:47,296: INFO: common: Created directory at : artifacts/model_trainer]
